In [1]:
import tensorflow as tf 

In [2]:
import os
import urllib.request as request

# Step 1: Download the data.
url = 'http://research.larc.smu.edu.sg/urldetection/static/samples/'


def maybe_download(filename):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = request.urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  print(statinfo.st_size)
  return filename

filename = maybe_download('urlsample.txt')

191876659


In [5]:
import re

# Read the data into a list of strings.
def read_data(filename):
    urls = []
    with open(filename) as f:
       lines = f.readlines()
       for line in lines:
           try:
               test_url = re.split(r'\t+', line)[1][:-1]
               urls.append(test_url)
           except IndexError:
               print("Wrong URL: ", line)
               continue
    return urls


urls = read_data(filename)
print('Data size', len(urls))


Wrong URL:  &amp;.rand=13vqcr8bp0gud&amp;lc=1033&amp;id=64855&amp;mkt=en-us&amp;cbcxt=mai&amp;snsc=1"

Wrong URL:  &amp;.rand=13vqcr8bp0gud&amp;lc=1033&amp;id=64855&amp;mkt=en-us&amp;cbcxt=mai&amp;snsc=1"

Data size 2558816


In [11]:
url_chars = []
for url in urls: 
    chars = list(url)
    url_chars.extend(chars)

In [30]:
print("Len by char: ", len(url_chars))
print("Len by url: ", len(urls))

Len by char:  181641215
Len by url:  2558816


In [15]:
import collections

# Step 2: Build the dictionary and replace rare words with UNK token.
vocabulary_size = 500

def build_dataset(words, vocabulary_size):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(url_chars, vocabulary_size)
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])



Most common words (+UNK) [['UNK', 0], ('t', 15694498), ('/', 12949094), ('e', 8642312), ('s', 8244125)]
Sample data [33, 1, 12, 28, 2, 2, 17, 21, 21, 13] ['f', 't', 'p', ':', '/', '/', '1', '8', '8', '.']


In [31]:
print("no. of unique chars: ", len(reverse_dictionary))
vocabulary_size = len(reverse_dictionary)

no. of unique chars:  122


In [17]:
import numpy as np
import random

# Step 3: Function to generate a training batch for the skip-gram model.
data_index = 0

def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1  # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [skip_window]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  # Backtrack a little bit to avoid skipping words in the end of a batch
  data_index = (data_index + len(data) - span) % len(data)
  return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
  print(batch[i], reverse_dictionary[batch[i]],
        '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

1 t -> 33 f
1 t -> 12 p
12 p -> 1 t
12 p -> 28 :
28 : -> 2 /
28 : -> 12 p
2 / -> 2 /
2 / -> 28 :


In [50]:
# Step 4: Build and train a skip-gram model.
import math

batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 5       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():

  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=train_labels,
                     inputs=embed,
                     num_sampled=num_sampled,
                     num_classes=vocabulary_size))

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

  # Add variable initializer.
  init = tf.global_variables_initializer()

In [51]:
# Step 5: Begin training.
from six.moves import xrange

num_steps = 2000000

with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    init.run()
    print("Initialized")

    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(
            batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print("Average loss at step ", step, ": ", average_loss)
            average_loss = 0

        # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in xrange(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = "Nearest to %s:" % valid_word
                for k in xrange(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = "%s %s," % (log_str, close_word)
                print(log_str)

        if step%50000 == 0:
            final_embeddings = normalized_embeddings.eval()
            tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
            plot_only = vocabulary_size
            low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
            labels = [reverse_dictionary[i] for i in xrange(plot_only)]
            plot_with_labels(low_dim_embs, labels, filename="".join(("url2vec-0.1-",str(step),".png")))

    final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  64.1358795166
Nearest to ": z, 6, 5, #, k, â, m, š,
Nearest to D: Z, M, , x, 3, o, L, Œ,
Nearest to ¿: R, ‚, K, *, ©, &, ¯, ;,
Nearest to n: i, 8, ., *, H, 9, {, E,
Nearest to 0: `, Æ, ,, …, v, 2, Ã, s,
Nearest to ‚: K, -, ¿, k, p, a, L, ,
Nearest to ': {, ¯, 5, T, B, h, , l,
Nearest to V: d, , /, ´, p, u, 5, L,
Nearest to R: (, p, ¿, v, &, UNK, 8, Ð,
Nearest to d: V, (, @, 3, , h, p, Â,
Nearest to  : q, @, ?, †, A, B, 3, %,
Nearest to ?: r, ,  , !, \, T, b, ƒ,
Nearest to o: L, E, S, }, =, W, °, D,
Nearest to H: Â, ©, =, …, F, Q, n, †,
Nearest to B: &, 9, q, 1,  , c, E, ',
Nearest to 4: G, ,, Â, †, A, k, Z, S,


C:\Users\hungle\AppData\Local\Continuum\Anaconda3\envs\python3.5\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Average loss at step  2000 :  5.26683698618
Average loss at step  4000 :  4.27433005536
Average loss at step  6000 :  4.26698937273
Average loss at step  8000 :  4.24214654946
Average loss at step  10000 :  4.22870379448
Nearest to ": â, z, š, #, 6, k, 5, c,
Nearest to D: Z, M, x, 3, o, L, R, i,
Nearest to ¿: R, ‚, K, *, ©, &, L, ;,
Nearest to n: i, ., d, 8, l, /, o, 9,
Nearest to 0: 2, v, s, h, Q, :, W, T,
Nearest to ‚: K, -, L, k, p, a, 6, =,
Nearest to ': {, ¯, , 5, ¢, ’, ), T,
Nearest to V: d, /, p, 5, u, r, L, n,
Nearest to R: (, p, 8, v, &, x, i, D,
Nearest to d: V, n, h, p, 3, (, l, s,
Nearest to  : q, @, ?, †, A, B, %, W,
Nearest to ?: r, m, b, \, T, ., t, l,
Nearest to o: L, E, S, i, =, n, a, D,
Nearest to H: =, Q, n, ©, F, Â, a, w,
Nearest to B: 9, &, 1, c, q, E, m, 6,
Nearest to 4: 8, 7, ., G, 3, 2, w, k,
Average loss at step  12000 :  4.25749927127
Average loss at step  14000 :  4.23752692068
Average loss at step  16000 :  4.23222426486
Average loss at step  18000 :  4.256

In [39]:
# Step 6: Visualize the embeddings.
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
    assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
    plt.figure(figsize=(40, 40))  # in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        plt.annotate(label,
                     xy=(x, y),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')

    plt.savefig(filename)

In [45]:
tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
plot_only = vocabulary_size
low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
labels = [reverse_dictionary[i] for i in xrange(plot_only)]
plot_with_labels(low_dim_embs, labels, filename="".join(("url2vec2000000.png")))

C:\Users\hungle\AppData\Local\Continuum\Anaconda3\envs\python3.5\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [46]:
len(final_embeddings)

122

In [47]:
len(labels)

122

In [35]:
print(final_embeddings[:5], labels[:5])

[[  1.09693430e-01   1.90923121e-02   8.89237598e-03  -9.12648886e-02
    9.89127308e-02  -1.29088402e-01   8.66665877e-03  -1.01243883e-01
   -1.31039202e-01  -7.35624060e-02  -1.23370469e-01  -6.05564914e-04
    6.03334345e-02   4.33883332e-02   8.53326619e-02   1.58337697e-01
   -1.60032928e-01   8.29951316e-02   1.89597365e-02   7.29780421e-02
   -4.91956957e-02   1.14263095e-01  -4.86182570e-02  -6.13733381e-02
   -2.43748091e-02   1.02300726e-01  -1.24407992e-01   3.16287577e-02
   -2.74565797e-02   6.18777424e-02  -1.25284851e-01  -2.01462992e-02
    1.59645692e-01   1.21223301e-01   1.08656533e-01  -9.04804692e-02
   -1.05627105e-01   1.21519431e-01  -1.31856754e-01  -1.34518459e-01
    1.55744374e-01   1.50815904e-01   3.78736258e-02   3.49788852e-02
    8.88711885e-02   8.79040435e-02  -1.12346761e-01  -3.66950445e-02
    2.41516344e-02   2.33177301e-02  -1.35078531e-04   5.31477109e-03
    1.27071425e-01  -1.08586393e-01   2.47703437e-02  -1.40328556e-01
    1.29004255e-01  

In [48]:
len(final_embeddings[0])

128